## elasticsearch port
elasticsearch http://elastic.nccu.asia/

In [1]:
import csv
import os
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch.client import IndicesClient

In [2]:
# es = Elasticsearch(["http://elastic.nccu.asia:80/"])
es=Elasticsearch(["http://localhost:9200/"])
es_index=IndicesClient(es)

In [3]:
es

<Elasticsearch([{'port': 9200, 'host': 'localhost', 'scheme': 'http'}])>

## 建 index
使用 smartcn 分詞器
#### mapping 
參照此表 https://goo.gl/5iPf53

<!-- 
es mapping intro https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html 
date type https://goo.gl/fVsyNk
number type https://www.elastic.co/guide/en/elasticsearch/reference/1.4/mapping-core-types.html
array type https://www.elastic.co/guide/en/elasticsearch/reference/1.4/mapping-array-type.html
geo type https://www.elastic.co/guide/en/elasticsearch/reference/1.4/mapping-geo-point-type.html
-->

In [4]:
properties={
'platform':{ 'type': 'string'  },
'content':{ 'type': 'string'  },
'uri':{ 'type': 'string'  },
'time':{'type':   'date',
        'format': "strict_date_optional_time||epoch_millis"  },
'content_language':{ 'type': 'string'  },
'comments_count':{ 'type': 'integer'  },
'event':{ 'type': 'string'  },
'favorite_count':{ 'type': 'integer'  },
'from_parent_id':{ 'type': 'string'  },## 用 string 即可(原設計 array)，參考 https://goo.gl/xw89hR
'media_id':{ 'type': 'string'  },
'media_name':{ 'type': 'string'  },
'media_like_count':{ 'type': 'integer'  },
'from_user_category':{ 'type': 'string'  },
'from_user_id':{ 'type': 'string'  },
'from_user_like_count':{ 'type': 'integer'  },
'from_user_language':{ 'type': 'string'  },
'from_user_location':{ 'type': 'string'  },
'from_user_name':{ 'type': 'string'  },
'from_user_nick':{ 'type': 'string'  },
'from_user_realname':{ 'type': 'string'  },
'geo':{ 'type': 'geo_point'  },
'twi_hashtags':{ 'type': 'string'  },# array
'fb_hashtags':{ 'type': 'string'  },# array
'like_count':{ 'type': 'integer'  },
'post_type':{ 'type': 'string'  },
'share_target_uri':{ 'type': 'string'  },# array
'share_target_title':{ 'type': 'string'  },# array
'shared_domain':{ 'type': 'string'  },
'shared_initial_uri':{ 'type': 'string'  },
'shared_title':{ 'type': 'string'  },# array
'shared_uri':{ 'type': 'string'  },# array
'shares_count':{ 'type': 'integer'  },
'status_type':{ 'type': 'string'  },
'story_tag':{ 'type': 'string'  },#
'tags_id':{ 'type': 'string'  },#
'tags_name':{ 'type': 'string'  },#
'title':{ 'type': 'string'  },
'domain':{'type':'string'},
'update_time':{'type':   'date',
               'format': "strict_date_optional_time||epoch_millis"  },

                }

In [5]:
es.indices.create(
    index="platform",
    body={
      'settings': {
        'number_of_shards': 5,
        'number_of_replicas': 1,
        'analysis': {
          "analyzer":{"default":{"type": "cjk"}}
          }
        },
      'mappings':{
         'twitter':{           
            'properties':properties
            }   
        }  
      },
    ignore=400
)

{'error': {'index': 'platform',
  'reason': 'already exists',
  'root_cause': [{'index': 'platform',
    'reason': 'already exists',
    'type': 'index_already_exists_exception'}],
  'type': 'index_already_exists_exception'},
 'status': 400}

## 看到所有欄位 還有狀態

In [6]:
es_index.get_mapping(index="platform",doc_type="twitter")

{}

## 把 index 清掉

In [8]:
es.indices.delete(index="platform",ignore=[400,404])

{'acknowledged': True}